
# <span style="color:#006E7F">__Introduction to Oxford Nanopore Data Analysis__ <a class="anchor"></span>  


Created by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and G. Sarah (AGAP-INRAE) - Septembre 2021 Formation SouthGreen

Adapted by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) - Novembre 2022
    
Adapted by J. Orjuela (DIADE-IRD) - Mai 2023

# <span style="color:#006E7F">__TP1 - BASECALLING and QC__ <a class="anchor" id="data"></span>  
    
## <span style="color: #4CACBC;"> 1. Creating the folder, downloading data and so on</span>  


### <span style="color: #4CACBD;"> 1.1 Real data </span>

Before starting, please download special data created for this practical training. Data are available on the  from I-Trop server.

Each participant will analyse a alguea samples from this PAPER??? [paper.](https://www.biorxiv.org/content/10.1101/2021.07.04.451066v1.full)


In [11]:
cd ~/work
mkdir -p DATA
cd DATA
# download your compressed algae 4222  
#wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/algae_data/4222_RB2.fastq.gz

Downloading available reference genome fasta file from XX. We will use this reference genome to compare with our results.

In [6]:
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/algae_data/G11_RB6_FLYE_MEDAKA_PILON_RAGTAG_GC.fasta

--2023-05-18 14:38:51--  https://itrop.ird.fr/algae_data/G11_RB6_FLYE_MEDAKA_PILON_RAGTAG_GC.fasta
Resolving itrop.ird.fr (itrop.ird.fr)... 91.203.35.184
Connecting to itrop.ird.fr (itrop.ird.fr)|91.203.35.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15487955 (15M)
Saving to: ‘G11_RB6_FLYE_MEDAKA_PILON_RAGTAG_GC.fasta’

G11_RB6_FLYE_MEDAKA 100%[===================>]  14.77M  1.12MB/s    in 11s     

2023-05-18 14:39:03 (1.37 MB/s) - ‘G11_RB6_FLYE_MEDAKA_PILON_RAGTAG_GC.fasta’ saved [15487955/15487955]

FINISHED --2023-05-18 14:39:03--
Total wall clock time: 12s
Downloaded: 1 files, 15M in 11s (1.37 MB/s)


In [12]:
mkdir -p REF
cd REF

In [13]:
mv ../G11_RB6_FLYE_MEDAKA_PILON_RAGTAG_GC.fasta REF.fasta

## <span style="color: #4CACBC;"> 2. Basecalling </span>  

When DNA molecules are sequenced, electrical signals are stocked on fast5 format files.

These signals need to be converted on standard fastq files to post-analysis.

Several training dataset models are usually used to convert fast5 to fastq. 

### <span style="color: #4CACBD;"> 2.1 Basecalling with Guppy  <span style="color:red"> (Don't run it! ) </span> </span>

Guppy is a data processing toolkit that contains the 'Oxford Nanopore Technologies' basecalling algorithms, and several bioinformatic post-processing features.

Basecalling with guppy can be launch using guppy tool. 

Guppy takes fast5 raw read files and transform electrical signal in fastq files.

We recommend to basecaller yours dataset using a GPU graphic card to obtain results quickly.

Documentation about how to run Guppy on I-trop GPU can be found on https://bioinfo.ird.fr/index.php/tutorials-fr/gpu/


In [1]:
MODEL="dna_r9.4.1_450bps_hac.cfg"
INPUT="path/to/FAST5"
OUTPUT="/path/to/FASTQ"
echo "guppy_basecaller -c ${MODEL} -i ${INPUT} --recursive -s ${OUTPUT} --num_callers 8 --gpu_runners_per_device 8 --device auto --min_qscore 7 --compress_fastq"

guppy_basecaller -c dna_r9.4.1_450bps_hac.cfg -i path/to/FAST5 --recursive -s /path/to/FASTQ --num_callers 8 --gpu_runners_per_device 8 --device auto --min_qscore 7 --compress_fastq


## <span style="color: #4CACBC;"> 3. Quality Control on Long Reads </span>  


Calculating data quantity

In [14]:
cd ~/work/DATA
pwd

/home/jovyan/work/DATA


Calculating how many bases were sequenced

In [18]:
seqtk seq -A /home/jovyan/work/DATA/4222_RB2.fastq.gz | grep -v ">" | wc -m

746854316


In [19]:
seqtk seq -A /home/jovyan/work/DATA/REF/REF.fasta | grep -v ">" |wc -m

15333808


### What is the sequencing depth if REF.fasta lenght is about 15.3Mb (15333808pb) ?

### <span style="color: #4CACBD;"> 3.1 Quality Control of FASTQ with Nanoplot (Hh real data --fastq ) </span>


Control reads quality using Nanoplot. You can parameter this tool using --help.

In [ ]:
NanoPlot --help

Check quality using NanoPlot over Hh real data fastq

In [20]:
# create a folder to save results
mkdir -p ~/work/RESULTS
cd ~/work/RESULTS

In [23]:
time NanoPlot -t 8 --fastq ~/work/DATA/4222_RB2.fastq.gz --outdir NANOPLOT_4222


real	1m25.246s
user	1m10.348s
sys	0m1.987s


Observe NanoStats.txt file

In [24]:
cat NANOPLOT_4222/NanoStats.txt

General summary:         
Mean read length:               11,898.6
Mean read quality:                  12.7
Median read length:              7,828.0
Median read quality:                12.9
Number of reads:                62,763.0
Read length N50:                20,829.0
STDEV read length:              12,487.7
Total bases:               746,791,553.0
Number, percentage and megabases of reads above quality cutoffs
>Q5:	62763 (100.0%) 746.8Mb
>Q7:	62762 (100.0%) 746.8Mb
>Q10:	55006 (87.6%) 672.3Mb
>Q12:	40974 (65.3%) 521.9Mb
>Q15:	9351 (14.9%) 129.7Mb
Top 5 highest mean basecall quality scores and their read lengths
1:	20.1 (1894)
2:	19.9 (3008)
3:	19.5 (4601)
4:	19.4 (4374)
5:	19.4 (13533)
Top 5 longest reads and their mean basecall quality score
1:	142831 (16.3)
2:	125499 (13.6)
3:	121149 (16.1)
4:	118209 (8.8)
5:	116238 (14.6)


## Observe `report.html` file

* What do you think about data? 

* What about reads quality qscore?

### <span style="color: #4CACBD;"> Others complemental tools </span>
 

https://github.com/wdecoster/NanoPlot#companion-scripts

* NanoComp: comparing multiple runs

* NanoStat: statistic summary report of reads or alignments

* NanoFilt: filtering and trimming of reads

* NanoLyse: removing contaminant reads (e.g. lambda control DNA) from fastq

* FiltLong : filtering long reads by quality https://github.com/rrwick/Filtlong
